In [127]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from datetime import datetime
from pyspark.sql.functions import to_timestamp


sc = SparkContext.getOrCreate()

In [128]:
user = [
    (0, "fs10", "Argentina",0),
    (1, "viole", "Argentina",1),
    (2, "fede", "usa","ciudad1", 2),
    (3, "seba", "japon","ciudad1", 3),
    (4, "luis", "usa","ciudad1", 4),
    (5, "naty", "Argentina","ciudad2",4)
]

publicacion = [
    (0, 0, "foto","#aguante_boca #sosdelaB #perro",0),
    (1, 1, "foto","#hola #perro #",1),
    (2, 2, "foto","#no_deberia_Estar", 2),
    (3, 2, "foto","#no_deberia_Estar", 3),
    (4, 2, "foto","#no_deberia_Estar", 4),
    (5, 1, "foto","#si_deberia_Estar #no_me_falles #perro",4),
    (6, 3, "foto","#no_deberia_Estar",4),
    (7, 4, "foto","#no_deberia_Estar",4),
    (8, 5, "foto","#van_a_Reprobar #sosdelaB",4),
    (9, 5, "foto"," sin hashtag",4)
]
     
     
rdd_user = sc.parallelize(user)
rdd_publicacion = sc.parallelize(publicacion)
     

In [129]:
def hashtag(s):
    lista = []
    palabras = s.split(' ')
    for palabra in palabras:
        if len(palabra) > 1 and palabra[0] == '#':
            lista.append(palabra.strip('#'))
    return lista

In [130]:
rdd_user_filter = rdd_user.filter(lambda x: x[2] == 'Argentina').map(lambda x: (x[0],1))
rdd_publicacion.map(lambda x: (x[1],x[3])).join(rdd_user_filter)\
.flatMap(lambda x: hashtag(x[1][0]))\
.map(lambda x: (x,1))\
.reduceByKey(lambda x,y: x + y)\
.takeOrdered(3, lambda x: -x[1])



[('perro', 3), ('sosdelaB', 2), ('si_deberia_Estar', 1)]